<a href="https://colab.research.google.com/github/rwcitek/anl-nerde/blob/main/nerde_argonne_api_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NERDE API via R



NERDE ( National Economic Resilience Data Explorer )



- https://www.anl.gov/dis/national-economic-resilience-data-explorer-nerde
- https://nerde.anl.gov/
- [Introduction to Nerde]( https://youtu.be/FCzSLtNCG5U?feature=shared ) on YouTube
- [API Docs]( https://nerde.anl.gov/resources/api-documentation )
  - [account registration/sign in]( https://nerde.anl.gov/auth/login )


Other resources:
- [Federal Information Processing Standards (FIPS) ]( https://en.wikipedia.org/wiki/Federal_Information_Processing_Standards )

## Tapping into the NERDE API



Welcome to the NERDE API for R users. Make sure that prior to attempting to connect to the NERDE API, ensure you have registered as a NERDE user at https://nerde.anl.gov/auth/login . The credentials used for your account will be the values for your Username and Password.

Table of Contents:

- Obtaining an API Token
- County Summary Data
- Demographics
- Housing
- Workforce
- Local Economy
- Industry Trends
- Risk & Resilience




### Obtaining an API Token


In [ ]:
## Remember to replace your username and password below
nerde_username <- "username@email.com"
nerde_password <- "password"

In [ ]:
library(jsonlite)
library(httr)

call <- "https://nerde.anl.gov"

# Pull API Token
token <- fromJSON( #transforms content to JSON
  content( #retrieves POST content
  POST(url=paste0(call,"/api/auth/login"),
       ## Remember to replace your username and password below
       body=list(userName=nerde_username, password=nerde_password, rememberMe=TRUE),
       encode = "json"),
     "text",
     encoding = "UTF-8"))

# Your unique key
key <- token$result$token

# Establishing fips and start and end dates for collection (start and end dataes are not often used)
my_fips <- "24011,24013,24015,24017,24019,24021,24023,24025,24027,24029,24031,24033,24035,24039,24037,24041,24043,24045,24047"
fips_length <- lengths(regmatches(my_fips,gregexpr(",",my_fips)))+1
my_start <- "2020-01-01"
my_end <- "2023-01-01"




In [ ]:
# r Retrieve Results function
NERDE_API <- function(section_url,fips){
  results_summary <- content(GET(url=paste0(call,section_url,"?county_fips=",fips),
    add_headers(key)))

results_summary_digest <- list()
## Each county will populate a list, this loops through each called county to parse results
for (i in 1:length(results_summary$result)){
x <- results_summary$result[[i]]
results_summary_digest <- c(results_summary_digest,x)
}
return(results_summary_digest)
}


### County Summary Data



In [ ]:
# r County Summary
section_url <- "/api/v1/county/explorer/summary"

summary_digest <- NERDE_API(section_url = section_url,fips=my_fips)
## Creating a data frame for special needs characteristics

# Remove EDCI and Local 24 month Unemployment
sp_need_vars <- c(grep("local",summary_digest),grep("local",summary_digest)-1)

# Unnesting API results
sp_need <- data.frame(stack(summary_digest[-sp_need_vars]))
sp_need$ind <- as.character(sp_need$ind)

#initial data frame for loop (does not pull in coal community information)
mycol <- unique(sp_need$ind)[1]
special_needs <- data.frame(mycol=sp_need[sp_need$ind==mycol,1])

# loop to populate data frame with special needs characteristics
for (i in 2:length(unique(sp_need$ind))){
  mycol <- unique(sp_need$ind)[i]
  l <- data.frame(sp_need[sp_need$ind==mycol,1])
  names(l) <- mycol
  ifelse(nrow(l)==fips_length,
         special_needs <- cbind(special_needs,l),

         #coal community data is a special case due to the fact that those communities that are not designated as coal communities do not pull information for this field. The following code works to identify those counties and accurately describe them as coal communities or not.
         {
         f <- data.frame(sp_need[sp_need$ind %in% c("county",mycol),1])
         cc <- grepl("County",f$sp_need.sp_need.ind..in..c..county...mycol...1.)
         cc <- !c(cc[-1],TRUE)
         cc <- cbind(f,cc)
         names(cc) <- c("county",mycol)
         cc <- cc[grepl("County",cc$county)==TRUE,]
         special_needs <- cbind(special_needs,cc[,mycol])
         names(special_needs)[ncol(special_needs)] <- mycol
         }
         )


}


In [ ]:
# r 24 month unemployment
# 24 month unemployment data cleaning - unemployment data is calculated in aggregate for the selection so all of your selected counties will be summarized for a single month

unemp_24mo_raw <- summary_digest$local_24month_unemployment_rate_series
m <- unlist(unemp_24mo_raw)
unemp_24mo <-stack(m)

timestamps <- unemp_24mo[unemp_24mo$ind=="timestamp",1]
values <-unemp_24mo[unemp_24mo$ind=="value",1]
values <- c(rep(NA,length(timestamps)-length(values)),values)

unemp_24mo <- data.frame(timestamps,values)


### Demographics



In [ ]:
demographics_digest <- NERDE_API(section_url = "/api/v1/county/explorer/demographics", fips=my_fips)

#print listed variables that need a seprate ingest function
unique(names(demographics_digest[sapply(demographics_digest,is.list)==TRUE]))
# Unnesting API results

demographics_digest <- data.frame(stack(demographics_digest[sapply(demographics_digest,is.list)==FALSE]))
demographics_digest$ind <- as.character(demographics_digest$ind)

mycol <- unique(demographics_digest$ind)[1]
demographics <- data.frame(mycol=demographics_digest[demographics_digest$ind==mycol,1])

# loop to populate data frame with special needs characteristics
for (i in 2:length(unique(demographics_digest$ind))){
  mycol <- unique(demographics_digest$ind)[i]
  l <- data.frame(demographics_digest[demographics_digest$ind==mycol,1])
  names(l) <- mycol
  demographics <- cbind(demographics,l)
         }


### Housing



In [ ]:
housing_digest <- NERDE_API(section_url = "/api/v1/county/explorer/housing", fips=my_fips)
unique(names(housing_digest[sapply(housing_digest,is.list)==TRUE]))
# Unnesting API results

housing_digest <- data.frame(stack(housing_digest[sapply(housing_digest,is.list)==FALSE]))
housing_digest$ind <- as.character(housing_digest$ind)

mycol <- unique(housing_digest$ind)[1]
housing <- data.frame(mycol=housing_digest[housing_digest$ind==mycol,1])

# loop to populate data frame
for (i in 2:length(unique(housing_digest$ind))){
  mycol <- unique(housing_digest$ind)[i]
  l <- data.frame(housing_digest[housing_digest$ind==mycol,1])
  names(l) <- mycol
  housing <- cbind(housing,l)
}


### Workforce



In [ ]:
workforce_digest <- NERDE_API(section_url = "/api/v1/county/explorer/workforce", fips=my_fips)
unique(names(workforce_digest[sapply(workforce_digest,is.list)==TRUE]))
# Unnesting API results

workforce_digest <- data.frame(stack(workforce_digest[sapply(workforce_digest,is.list)==FALSE]))
workforce_digest$ind <- as.character(workforce_digest$ind)

mycol <- unique(workforce_digest$ind)[1]
workforce <- data.frame(mycol=workforce_digest[workforce_digest$ind==mycol,1])

# loop to populate data frame
for (i in 2:length(unique(workforce_digest$ind))){
  mycol <- unique(workforce_digest$ind)[i]
  l <- data.frame(workforce_digest[workforce_digest$ind==mycol,1])
  names(l) <- mycol
  workforce <- cbind(workforce,l)
}


### Local Economy



In [ ]:
l_econ_digest <- NERDE_API(section_url = "/api/v1/county/explorer/localeconomy", fips=my_fips)
unique(names(l_econ_digest[sapply(l_econ_digest,is.list)==TRUE]))
# Unnesting API results

l_econ_digest <- data.frame(stack(l_econ_digest[sapply(l_econ_digest,is.list)==FALSE]))
l_econ_digest$ind <- as.character(l_econ_digest$ind)

mycol <- unique(l_econ_digest$ind)[1]
local_econ <- data.frame(mycol=l_econ_digest[l_econ_digest$ind==mycol,1])

# loop to populate data frame
for (i in 2:length(unique(l_econ_digest$ind))){
  mycol <- unique(l_econ_digest$ind)[i]
  l <- data.frame(l_econ_digest[l_econ_digest$ind==mycol,1])
  names(l) <- mycol
  local_econ <- cbind(local_econ,l)
}


### Industry Trends



In [ ]:
industry_digest <- NERDE_API(section_url = "/api/v1/county/explorer/industrytrends", fips=my_fips)
unique(names(industry_digest[sapply(industry_digest,is.list)==TRUE]))

# Unnesting API results

industry_digest <- data.frame(stack(industry_digest[sapply(industry_digest,is.list)==FALSE]))
industry_digest$ind <- as.character(industry_digest$ind)

mycol <- unique(industry_digest$ind)[1]
industry <- data.frame(mycol=industry_digest[industry_digest$ind==mycol,1])

# loop to populate data frame
for (i in 2:length(unique(industry_digest$ind))){
  mycol <- unique(industry_digest$ind)[i]
  l <- data.frame(industry_digest[industry_digest$ind==mycol,1])
  names(l) <- mycol
  industry <- cbind(industry,l)
}


### Risk and Resilience



In [ ]:
risk_digest <- NERDE_API(section_url = "/api/v1/county/explorer/riskresilience", fips=my_fips)
unique(names(risk_digest[sapply(risk_digest,is.list)==TRUE]))

# Unnesting API results

risk_digest <- data.frame(stack(risk_digest[sapply(risk_digest,is.list)==FALSE]))
risk_digest$ind <- as.character(risk_digest$ind)

mycol <- unique(risk_digest$ind)[1]
risk <- data.frame(mycol=risk_digest[risk_digest$ind==mycol,1])

# loop to populate data frame with special needs characteristics
for (i in 2:length(unique(risk_digest$ind))){
  mycol <- unique(risk_digest$ind)[i]
  l <- data.frame(risk_digest[risk_digest$ind==mycol,1])
  names(l) <- mycol
  risk <- cbind(risk,l)
}
